# Model Analysis and Visualization

This notebook provides tools for downloading and analyzing trained models from Weights & Biases.

In [2]:
%load_ext autoreload
%autoreload 2

# Import required libraries
from pathlib import Path

# Import our wandb utilities
from fsrl.utils.wandb_utils import (
    WandBModelDownloader,
    download_model_family,
    list_model_family
)

import pandas as pd

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Current Model Status

Let's first check what models we currently have downloaded locally.

In [3]:
# Test path resolution to ensure models go to the right place
test_downloader = WandBModelDownloader(
    entity="feature-steering-RL",
    project="Gemma2-2B"
)

print(f"Current working directory: {Path.cwd()}")
print(f"Models will be saved to: {test_downloader.models_base_dir}")
print(f"Gemma2-2B models will go to: {test_downloader.models_base_dir / 'Gemma2_2B'}")

# Verify this is correct
expected_path = Path.cwd().parent / "models"
print(f"Expected path: {expected_path}")
print(f"Paths match: {test_downloader.models_base_dir == expected_path}")

Current working directory: /home/jazhyc/projects/FSRL/feature-steering-RL/notebooks
Models will be saved to: /home/jazhyc/projects/FSRL/feature-steering-RL/models
Gemma2-2B models will go to: /home/jazhyc/projects/FSRL/feature-steering-RL/models/Gemma2_2B
Expected path: /home/jazhyc/projects/FSRL/feature-steering-RL/models
Paths match: True


In [4]:
# Check what Gemma2 models we currently have downloaded
current_models = list_model_family()
print(f"Currently downloaded Gemma2-2B models: {len(current_models)}")
for model in current_models:
    print(f"  - {model}")

# Also check the models directory structure
models_dir = Path("../models")
print(f"\nModels directory structure:")
for item in models_dir.iterdir():
    if item.is_dir():
        model_count = len([x for x in item.iterdir() if x.is_dir()])
        print(f"  {item.name}/: {model_count} models")

Currently downloaded Gemma2-2B models: 0

Models directory structure:
  gpt2-debug/: 7 models
  test/: 0 models


## Explore Available WandB Runs

Let's check what runs are available in the Gemma2-2B project on Weights & Biases.

In [5]:
# Initialize the downloader
downloader = WandBModelDownloader(
    entity="feature-steering-RL",
    project="Gemma2-2B"
)

# Get completed runs
completed_runs = downloader.get_completed_runs()
print(f"Found {len(completed_runs)} completed runs")

# Create a summary DataFrame
run_data = []
for run in completed_runs:
    artifacts = downloader.get_run_artifacts(run)
    run_data.append({
        'run_name': run.name,
        'state': run.state,
        'created_at': run.created_at,
        'finished_at': getattr(run, 'finished_at', None),
        'num_artifacts': len(artifacts),
        'artifacts': [a.name for a in artifacts],
        'already_downloaded': downloader.is_model_downloaded("gemma2_2B-fsrl", run.name)
    })

# Convert to DataFrame for better visualization
runs_df = pd.DataFrame(run_data)
print(f"\nRun Summary:")
print(runs_df[['run_name', 'state', 'num_artifacts', 'already_downloaded']].to_string(index=False))

# Show details for runs with artifacts
print(f"\nRuns with model artifacts:")
for _, row in runs_df[runs_df['num_artifacts'] > 0].iterrows():
    status = "✓ Downloaded" if row['already_downloaded'] else "⏳ Not downloaded"
    print(f"  {row['run_name']}: {row['num_artifacts']} artifacts {status}")
    for artifact in row['artifacts']:
        print(f"    - {artifact}")

INFO:fsrl.utils.wandb_utils:Found 2 completed runs in feature-steering-RL/Gemma2-2B


Found 2 completed runs

Run Summary:
           run_name    state  num_artifacts  already_downloaded
hearty-wildflower-1 finished              1               False
  ethereal-salad-14 finished              1               False

Runs with model artifacts:
  hearty-wildflower-1: 1 artifacts ⏳ Not downloaded
    - adapter-hearty-wildflower-1:v0
  ethereal-salad-14: 1 artifacts ⏳ Not downloaded
    - adapter-ethereal-salad-14:v0

Run Summary:
           run_name    state  num_artifacts  already_downloaded
hearty-wildflower-1 finished              1               False
  ethereal-salad-14 finished              1               False

Runs with model artifacts:
  hearty-wildflower-1: 1 artifacts ⏳ Not downloaded
    - adapter-hearty-wildflower-1:v0
  ethereal-salad-14: 1 artifacts ⏳ Not downloaded
    - adapter-ethereal-salad-14:v0


## Download Models

Now let's download all the completed models that haven't been downloaded yet.

In [6]:
# Download all Gemma2 models (only downloads those not already present)
print("Starting model download process...")
download_results = download_model_family(force_download=False)

# Show results
successful_downloads = sum(1 for path in download_results.values() if path is not None)
print(f"\nDownload Results: {successful_downloads}/{len(download_results)} models downloaded")

for run_name, path in download_results.items():
    if path is not None:
        print(f"✓ {run_name}: {path}")
    else:
        print(f"✗ {run_name}: No artifacts or download failed")

Starting model download process...


INFO:fsrl.utils.wandb_utils:Found 2 completed runs in feature-steering-RL/Gemma2-2B
INFO:fsrl.utils.wandb_utils:Processing run: hearty-wildflower-1 (state: finished)
INFO:fsrl.utils.wandb_utils:Processing run: hearty-wildflower-1 (state: finished)
INFO:fsrl.utils.wandb_utils:Downloading artifact adapter-hearty-wildflower-1:v0 from run hearty-wildflower-1
INFO:fsrl.utils.wandb_utils:Downloading artifact adapter-hearty-wildflower-1:v0 from run hearty-wildflower-1
wandb:   3 of 3 files downloaded.  
INFO:fsrl.utils.wandb_utils:Downloaded to /home/jazhyc/projects/FSRL/feature-steering-RL/models/Gemma2-2B/hearty-wildflower-1
INFO:fsrl.utils.wandb_utils:Successfully downloaded 1 artifacts for run hearty-wildflower-1
INFO:fsrl.utils.wandb_utils:Processing run: ethereal-salad-14 (state: finished)
wandb:   3 of 3 files downloaded.  
INFO:fsrl.utils.wandb_utils:Downloaded to /home/jazhyc/projects/FSRL/feature-steering-RL/models/Gemma2-2B/hearty-wildflower-1
INFO:fsrl.utils.wandb_utils:Successful


Download Results: 2/2 models downloaded
✓ hearty-wildflower-1: /home/jazhyc/projects/FSRL/feature-steering-RL/models/Gemma2-2B/hearty-wildflower-1
✓ ethereal-salad-14: /home/jazhyc/projects/FSRL/feature-steering-RL/models/Gemma2-2B/ethereal-salad-14


## Verify Downloads

Let's verify that the models were downloaded correctly and explore their structure.

In [12]:
# Check updated model list
updated_models = list_model_family()
print(f"Total Gemma2-2B models after download: {len(updated_models)}")

# Explore the structure of downloaded models
models_base_path = Path("../models/Gemma2-2B")
if models_base_path.exists():
    print(f"\nModel directory structure in {models_base_path}:")
    for model_dir in sorted(models_base_path.iterdir()):
        if model_dir.is_dir():
            files = list(model_dir.glob("*"))
            print(f"\n  📁 {model_dir.name}/")
            for file in sorted(files):
                if file.is_file():
                    size_mb = file.stat().st_size / (1024 * 1024)
                    print(f"    📄 {file.name} ({size_mb:.1f} MB)")
                elif file.is_dir():
                    sub_files = len(list(file.glob("*")))
                    print(f"    📁 {file.name}/ ({sub_files} files)")
else:
    print(f"Directory {models_base_path} does not exist yet.")

Total Gemma2-2B models after download: 2

Model directory structure in ../models/Gemma2-2B:

  📁 ethereal-salad-14/
    📁 adapter/ (3 files)

  📁 hearty-wildflower-1/
    📁 adapter/ (3 files)
